In [34]:
import requests

from sqlalchemy import create_engine
import pymysql
import db_connection as db_conn
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import pandas as pd
import numpy as np
import pickle

import random
from web3 import Web3
# choose a Provider of your choice

from alchemy_api import AlchemyApi
import concurrent.futures
import re

In [2]:
alchemy = AlchemyApi()

In [3]:
scam_df = pd.read_csv('data/hoptrail_scam_dataset.csv')
print(scam_df.shape[0])

8473


In [4]:
# get the scam addresses as list
scam_addresses = scam_df['address'].tolist()
original_scam_df = pd.read_csv('data/transaction_dataset.csv')
original_scam_df = original_scam_df[original_scam_df['FLAG'] == 1]
original_scam_addresses = original_scam_df['Address'].tolist()
filtered_original_scams = [scam for scam in original_scam_addresses if scam in scam_addresses]
filtered_missing_scams = [scam for scam in original_scam_addresses if scam not in scam_addresses]

print('Original scams: ', len(original_scam_addresses))
print('Original scams in scam_users: ', len(filtered_original_scams))
print('Filtered missing scams from scam_users: ', len(filtered_missing_scams))

scam_addresses = scam_addresses + filtered_missing_scams

Original scams:  2179
Original scams in scam_users:  1981
Filtered missing scams from scam_users:  198


In [5]:
def is_valid_ethereum_address(address):
    match = re.match('^0x[a-fA-F0-9]{40}$', address)
    return match is not None

def is_valid_ethereum_tx_hash(tx_hash):
    match = re.match('^0x[a-fA-F0-9]{64}$', tx_hash)
    return match is not None

invalid_addresses = []
for address in scam_addresses:
    if not is_valid_ethereum_address(address):
        invalid_addresses.append(address)
    if is_valid_ethereum_tx_hash(address):
        invalid_addresses.append(address)

invalid_addresses = list(set(invalid_addresses))
print(len(invalid_addresses))
# Remove invalid addresses from all_addresses list
scam_addresses = [address for address in scam_addresses if address not in invalid_addresses]
len(scam_addresses)

5


8666

In [6]:
with open('data/pickle_files/scam_users_and_contract_creators_alt.pkl', 'wb') as f:
    pickle.dump(scam_addresses, f)

In [2]:
scam_users = pd.read_pickle('data/pickle_files/scam_users_and_contract_creators_alt.pkl')
scam_users

['...',
 ...]

In [8]:
db_con_test = db_conn.config_sql
db_con_train = db_conn.config_train_sql
spark = SparkSession.builder \
    .appName("process_tx") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "32g") \
    .getOrCreate()

23/08/04 13:02:21 WARN Utils: Your hostname, NatRng-MBP.local resolves to a loopback address: 127.0.0.1; using 10.200.168.84 instead (on interface en0)
23/08/04 13:02:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/04 13:02:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/04 13:02:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
url = f"jdbc:mariadb://{db_con_test['host']}:{db_con_test['port']}/{db_con_test['database']}"
user = db_con_test['user']
password = db_con_test['password']
address_test_df = spark.read.format('jdbc').options(url=url, dbtable='Addresses', user=user, password=password).load()

url = f"jdbc:mariadb://{db_con_train['host']}:{db_con_train['port']}/{db_con_train['database']}"
user = db_con_train['user']
password = db_con_train['password']
address_train_df = spark.read.format('jdbc').options(url=url, dbtable='Addresses', user=user, password=password).load()

In [10]:
train_matches = address_train_df.filter(col('address').isin(scam_users))
test_matches = address_test_df.filter(col('address').isin(scam_users))

In [11]:
train_addresses = [row['address'] for row in train_matches.collect()]
test_addresses = [row['address'] for row in test_matches.collect()]
scam_addresses_in_train_test = list(set(train_addresses) | set(test_addresses))

In [12]:
with open('data/pickle_files/scam_addresses_in_train_test_alt.pkl', 'wb') as f:
    pickle.dump(scam_addresses_in_train_test, f)

In [6]:
training_data_full= pd.read_parquet('data/parquet_files/training_data_rfm_alt.parquet')
len(training_data_full)

79958

In [28]:
hoptrail_data = pd.read_csv('data/hoptrail_scam_dataset.csv')
contract_creators = pd.read_parquet('data/parquet_files/contract_creators.parquet')
contract_addresses_in_scam= [address for sublist in contract_creators['contract_address'].tolist() for address in sublist]
contract_addresses_in_scam
hoptrail_addresses = set(hoptrail_data['address'])
contract_addresses_set = set(contract_addresses_in_scam)
hoptrail_addresses_contracts = hoptrail_addresses.intersection(contract_addresses_set)

In [37]:
contract_scams = hoptrail_data[hoptrail_data['address'].isin(hoptrail_addresses_contracts)]

In [35]:
config_scam_sql = db_conn.config_scam_alt_sql
engine_scam_sql = create_engine(f"mysql+pymysql://{config_scam_sql['user']}:{config_scam_sql['password']}@{config_scam_sql['host']}:{config_scam_sql['port']}/{config_scam_sql['database']}")
scam_address_df = pd.read_sql('SELECT * FROM Addresses', con=engine_scam_sql)

In [45]:
contract_scam_ids_df = contract_scams.merge(scam_address_df, on='address', how='left')

In [50]:
contract_scam_ids_df.dropna(subset=['address_id'], inplace=True)
contract_scam_ids_df['address_id'] = contract_scam_ids_df['address_id'].astype(int)
contract_scam_ids_list = contract_scam_ids_df['address_id'].tolist()

In [54]:
scam_df = pd.read_parquet('data/parquet_files/account_df_scam_alt.parquet')
pd.set_option('display.max_columns', None)
contract_scam_train = scam_df[scam_df['address_id'].isin(contract_scam_ids_list)]
stats = contract_scam_train.describe()
median = contract_scam_train.median()
mad = contract_scam_train.mad()
stats.loc['median'] = median
stats.loc['mad'] = mad
stats

/var/folders/xs/x2x4yg29089c8t_v4z9nymz40000gn/T/ipykernel_10071/1067833828.py:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  median = contract_scam_train.median()
/var/folders/xs/x2x4yg29089c8t_v4z9nymz40000gn/T/ipykernel_10071/1067833828.py:6: FutureWarning: The 'mad' method is deprecated and will be removed in a future version. To compute the same result, you may do `(df - df.mean()).abs().mean()`.
  mad = contract_scam_train.mad()


,address_id,outgoing_tx_count,incoming_tx_count,unique_received_transactions,unique_sent_transactions,contracts_created,total_tx_with_contracts,total_incoming_eth,total_outgoing_eth,time_difference_in_minutes,avg_time_diff_out_minutes,avg_time_diff_in_minutes,total_avg_time_diff_minutes,outgoing_erc_20_tnx,incoming_erc_20_tnx,total_erc_20_tnx,total_outgoing_erc20eth,total_incoming_erc20eth,min_erc20token_in,min_erc20token_out,max_erc20token_in,max_erc20token_out,num_unique_erc20tokens_out,num_unique_erc20tokens_in,outgoing_erc_1155_tnx,incoming_erc_1155_tnx,total_erc_1155_tnx,outgoing_erc_721_tnx,incoming_erc_721_tnx,total_erc_721_tnx,num_unique_erc721asset_out,num_unique_erc721asset_in,daily_from_gini_index,daily_to_gini_index,weekly_from_gini_index,weekly_to_gini_index,daily_total_gini_index,weekly_total_gini_index,median_recency_out,median_recency_in,num_outliers_eth_out,num_outliers_eth_in
count,1.884000e+03,1.884000e+03,1.884000e+03,1884.000000,1884.000000,1884.0,1.884000e+03,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1.884000e+03,1884.000000,1.884000e+03,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.000000,1884.0,1.884000e+03,1.884000e+03,1884.0,1884.000000,628.000000,1883.000000,628.000000,1883.000000,1884.000000,1884.000000,1884.000000,1884.000000,1811.000000,1.811000e+03
mean,7.732240e+05,2.539151e+03,2.131725e+03,716.346072,581.599257,0.0,4.670876e+03,888.551811,859.090181,14575.275062,587.338102,2061.395924,1808.626681,2.352848e+03,0.616242,2.353464e+03,859.090186,415.079309,14.901110,13.726382,51.826065,62.892830,0.307325,0.063163,0.001062,0.001062,0.002123,0.0,2.124833e+03,2.124833e+03,0.0,0.955414,0.848422,0.868171,0.594043,0.590071,0.869217,0.592128,0.197221,0.559316,955.110988,4.016786e+03
std,6.654684e+05,4.171897e+04,3.580211e+04,12874.979703,8209.991094,0.0,7.744318e+04,18163.186072,17958.433086,16093.638291,2486.167550,4384.941876,4011.696982,4.158663e+04,5.330477,4.158808e+04,17958.433134,16287.440045,574.891357,572.086609,1317.597168,1346.114502,0.461508,0.243321,0.046078,0.032573,0.072843,0.0,3.580059e+04,3.580059e+04,0.0,0.206448,0.195781,0.166765,0.194305,0.195342,0.166320,0.194333,0.373494,0.432616,13441.979041,9.014119e+04
min,9.120000e+02,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.055851,0.053758,0.051902,0.000000,0.055726,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,1.010428e+05,0.000000e+00,2.000000e+00,1.000000,0.000000,0.0,2.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000e+00,1.000000e+00,0.0,1.000000,0.833333,0.850689,0.480141,0.450000,0.854970,0.451018,0.000000,0.000701,0.000000,0.000000e+00
50%,6.104995e+05,0.000000e+00,6.000000e+00,3.000000,0.000000,0.0,7.000000e+00,0.000000,0.000000,6624.733333,0.000000,138.476278,105.420755,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,5.000000e+00,5.000000e+00,0.0,1.000000,0.933333,0.933333,0.716667,0.718750,0.935249,0.720444,0.000000,0.683247,0.000000,0.000000e+00
75%,1.459574e+06,2.000000e+00,2.800000e+01,10.000000,1.000000,0.0,3.400000e+01,0.139882,0.117964,29342.450000,0.014990,2192.051875,1837.665404,1.000000e+00,0.000000,1.000000e+00,0.117964,0.000000,0.000000,0.000204,0.000000,0.069250,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.625000e+01,2.625000e+01,0.0,1.000000,0.966667,0.966667,0.750000,0.750000,0.966667,0.750000,0.018286,0.995919,0.000000,0.000000e+00
max,1.840542e+06,1.307578e+06,1.115378e+06,404508.000000,234661.000000,0.0

In [55]:
contract_scam_train

,address_id,year_month,outgoing_tx_count,incoming_tx_count,unique_received_transactions,unique_sent_transactions,contracts_created,total_tx_with_contracts,total_incoming_eth,total_outgoing_eth,time_difference_in_minutes,avg_time_diff_out_minutes,avg_time_diff_in_minutes,total_avg_time_diff_minutes,outgoing_erc_20_tnx,incoming_erc_20_tnx,total_erc_20_tnx,total_outgoing_erc20eth,total_incoming_erc20eth,min_erc20token_in,min_erc20token_out,max_erc20token_in,max_erc20token_out,num_unique_erc20tokens_out,num_unique_erc20tokens_in,outgoing_erc_1155_tnx,incoming_erc_1155_tnx,total_erc_1155_tnx,outgoing_erc_721_tnx,incoming_erc_721_tnx,total_erc_721_tnx,num_unique_erc721asset_out,num_unique_erc721asset_in,daily_from_gini_index,daily_to_gini_index,weekly_from_gini_index,weekly_to_gini_index,daily_total_gini_index,weekly_total_gini_index,median_recency_out,median_recency_in,num_outliers_eth_out,num_outliers_eth_in
15,4093,2019-06,1,3,2,1,0,4,0.00000,0.010898,20725.650000,0.000000,10362.825000,6908.550000,1,0,1,0.010898,0.0,0.0,1.089840e-02,0.0,0.010898,1,0,0,0,0,0,3,3,0,1,0.966667,0.900000,0.750000,0.583333,0.916667,0.500000,0.000000,0.020595,0.0,0.0
32,6656,2015-09,42,108,60,25,0,150,127.01021,109.020211,33675.716667,814.426423,314.726324,226.011521,42,25,67,109.020212,25.0,1.0,6.500000e-07,1.0,12.001200,1,1,0,0,0,0,83,83,0,1,0.701587,0.717284,0.464286,0.462963,0.712889,0.463333,0.961091,0.990098,0.0,0.0
33,6656,2016-01,44,118,55,21,0,162,116.98000,105.880000,40272.183333,936.562403,344.206695,250.137785,44,47,91,105.880002,47.0,1.0,1.000000e-18,1.0,2.700000,1,1,0,0,0,0,71,71,0,1,0.721212,0.729379,0.563636,0.616949,0.723457,0.602469,0.961710,0.996110,0.0,0.0
34,6656,2016-03,56,178,61,25,0,234,178.00405,147.800000,43285.283333,755.349394,244.549623,185.773748,56,10,66,147.800003,10.0,1.0,1.000000e+00,1.0,2.700000,1,1,0,0,0,0,168,168,0,1,0.760714,0.728839,0.550000,0.505618,0.733903,0.516239,1.000000,1.000000,0.0,0.0
35,6656,2018-07,0,1,1,0,0,1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.000000,0,0,0,0,0,0,0,0,0,0,NaN,0.966667,NaN,0.750000,0.966667,0.750000,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26626,1488157,2018-09,0,13,13,0,0,13,0.00000,0.000000,7497.683333,0.000000,624.806944,624.806944,0,0,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.000000,0,0,0,0,0,0,13,13,0,1,NaN,0.951282,NaN,0.750000,0.951282,0.750000,0.000000,0.999639,0.0,0.0
26629,1500182,2021-05,0,244,162,0,0,244,0.00000,0.000000,12752.033333,0.000000,52.477503,52.477503,0,0,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.000000,0,0,0,0,0,0,244,244,0,1,NaN,0.917486,NaN,0.700820,0.917486,0.700820,0.000000,0.999398,0.0,0.0
26630,1500182,2021-12,0,1,1,0,0,1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.000000,0,0,0,0,0,0,1,1,0,1,NaN,0.966667,NaN,0.750000,0.966667,0.750000,0.000000,0.000000,0.0,0.0
26634,1540300,2022-02,0,19,6,0,0,19,0.00000,0.000000,1757.666667,0.000000,97.648148,97.648148,0,0,0,0.000000,0.0,0.0,0.000000e+00,0.0,0.000000,0,0,0,0,0,0,19,19,0,1,NaN,0.949123,NaN,0.750000,0.949123,0.750000,0.000000,0.998068,0.0,0.0
